# Определение перспективного тарифа для телеком компании

Вы аналитик компании «Мегалайн» — федерального оператора сотовой связи. Клиентам предлагают два тарифных плана: «Смарт» и «Ультра».

**Тариф «Смарт»**

1. Ежемесячная плата: 550 рублей
2. Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
3. Стоимость услуг сверх тарифного пакета:
    - минута разговора: 3 рубля
    - сообщение: 3 рубля
    - 1 Гб интернет-трафика: 200 рублей

**Тариф «Ультра»**

1. Ежемесячная плата: 1950 рублей
2. Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
3. Стоимость услуг сверх тарифного пакета:
    - минута разговора: 1 рубль
    - сообщение: 1 рубль
    - 1 Гб интернет-трафика: 150 рублей

Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, **какой тариф приносит больше денег**.

В вашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год.

🎯 Нужно проанализировать поведение клиентов и сделать вывод — **какой тариф лучше**.

## Changelog

- При слиянии датафреймов используется параметр `how='outer'` вместо `right` для того, чтобы получить записи по ключам из обоих датасетов;
- Дисперсия и другие метрики считаются не по агрегированным данным. Описание шага 3 ввело в заблуждение;
- Скорректированы выводы.

## Шаг 1. Откроем файлы с данными и изучим общую информацию

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as st

In [ ]:
calls = pd.read_csv('calls.csv')
internet = pd.read_csv('internet.csv')
messages = pd.read_csv('messages.csv')
tariffs = pd.read_csv('tariffs.csv')
users = pd.read_csv('users.csv')

Для начала рассмотрим каждый из датасетов по отдельности. При необходимости будем соединять данные.

### Тарифы

In [ ]:
tariffs.info()

In [ ]:
'''
    Сразу же переименуем колонку с названием тарифа, чтобы в дальнейшем у нас была возможность применить merge по ней.
    Также исправим ошибку в названии признака объёма интернет-трафика.
'''
tariffs = tariffs.rename(columns={'tariff_name': 'tariff', 'mg_per_month_included': 'mb_per_month_included'})

In [ ]:
tariffs

С типами данных все ок.

📝 Объём интернет-трафика, включённого в абонентскую плату, указан в мегабайтах, а стоимость дополнительного объема интернет-трафика в рублях за гигабайт.

- Приведите данные к нужным типам;
- Найдите и исправьте ошибки в данных.

Поясните, какие ошибки вы нашли и как их исправили.
Обратите внимание, что у большого количества звонков длительность — 0.0 минут. Это проблема в данных, нужна предобработка.


162994

In [19]:
calls.duration.replace(0.0, np.nan, inplace=True)
calls.duration.fillna(calls.duration.mean(), inplace=True)

calls.call_date = pd.to_datetime(calls.call_date)

In [21]:
internet.mb_used.replace(0.0, np.nan, inplace=True)
internet.mb_used.fillna(internet.mb_used.mean(), inplace=True)

internet.session_date = pd.to_datetime(internet.session_date)

In [22]:
messages.message_date = pd.to_datetime(messages.message_date)

In [27]:
users.churn_date = pd.to_datetime(users.churn_date)
users.reg_date = pd.to_datetime(users.reg_date)

Посчитайте для каждого пользователя:

- количество сделанных звонков и израсходованных минут разговора по месяцам;
- количество отправленных сообщений по месяцам;
- объем израсходованного интернет-трафика по месяцам;
- помесячную выручку с каждого пользователя (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана).